# 提取页面元素
- 提取元素的作用：避免UI界面经常改变，影响自动化脚本，导致需要多次修改脚本文件，增加工作量
- 使用到的第三方库：
    - [yaml介绍](/tree/第三方库/yaml%20-%20配置文件.ipynb)
    - [jinja2介绍](/tree/第三方库/jinja2%20-%20模板.ipynb)
- 具体实现的步骤
    - 安装yaml和jinja2
    - 建立目录page_element：
        - element_desc：存放所有的yaml文件
        - page_elements.py：用于生成yaml文件中每个页面类
        - tools.py：用于执行，从而生成page_elemtns.py文件
        - template_page.txt：模板文件，用于按照这个格式生成page_elements.py文件

![](img/1.png)

In [ ]:
# yaml文件
LoginPage:
  dec: 登录页面
  locators:
    -
      desc: 选择页跳转链接
      name: login_text
      type: id
      value: com.liukena.android:id/logintext
    -
      desc: 未登录的首页 - 我的按钮
      name: me_btn
      type: id
      value: com.liukena.android:id/rbtn_me
    -
      desc: 未登录的首页 - 我的按钮 - 登录按钮
      name: me_ok_btn
      type: id
      value: com.liukena.android:id/ok_text
    -
      desc: 密码登录页面 - 手机号输入框
      name: phone_input
      type: id
      value: com.liukena.android:id/etwd_phone
    -
      desc: 密码登录页面 - 密码输入框
      name: pwd_input
      type: id
      value: com.liukena.android:id/etwd_password
    -
      desc: 密码登录页面 - 登录按钮
      name: login_btn
      type: id
      value: com.liukena.android:id/tv_login_immediately

In [ ]:
#tools.py文件
'''
    直接执行，通过模板生成页面元素文件 - page_elements
'''

import yaml
import os
import jinja2

basepath = os.path.dirname(__file__)

# yaml文件位置
yamlpath = os.path.join(basepath,'element_desc')

def parseyaml():
    '''
    遍历读取yaml文件
    :return:
    '''
    pageElements = {}
    for fpath,dirname,fnames in os.walk(yamlpath):
        for name in fnames:
            # yaml文件绝对路径
            yaml_file_path = os.path.join(fpath,name)
            # 排除非.yaml的文件
            if ".yaml" in str(yaml_file_path):
                with open(yaml_file_path,'r',encoding='utf-8') as f:
                    page = yaml.load(f)
                    pageElements.update(page)
    return pageElements


def get_page_list(yamlpage):
    '''
    对获取到的页面元素字典进行格式化
    :param yamlpage:通过上面的函数获取到的yaml数据字典
    :return:返回一个字典
        {'login':[{desc:xxx,name:xxx}]}
    '''
    page_object = {}
    for page,names in yamlpage.items():
        loc_names = []
        locs = names['locators']
        for loc in locs:
            loc_names.append({'desc':loc['desc'],'name':loc['name']})
        page_object[page] = loc_names
    return page_object


def create_pages_py(page_list):
    '''
    根据模板生成.py文件
    :param page_list:
    :return:
    '''
    template_loader = jinja2.FileSystemLoader(searchpath=basepath)
    template_env = jinja2.Environment(loader=template_loader)

    templateVars = {
        'page_list':page_list
    }
    template = template_env.get_template("template_page")
    with open(os.path.join(basepath,"page_elements.py"),'w',encoding='utf-8') as f:
        f.write(template.render(templateVars))

if __name__ == "__main__":
    '''执行'''
    # 获取yaml中所有内容
    page_element = parseyaml()
    # 对内容进行格式化
    page_list = get_page_list(page_element)
    # 根据
    create_pages_py(page_list)

In [ ]:
# template_page.txt文件
from tools import parseyaml
# 用于在获取获取元素的字符串，为什么不直接获取 -  防止改变字符串的时候需要重新生成该文件
pages = parseyaml()

# 获取
def get_locater(clazz_name,method_name):
    locators = pages[clazz_name]['loctors']
    for locator in locators:
        if locator['name'] == method_name:
            return locator


{% for page,locators in page_list.items() %}
class {{page}}:
    {% for locator in locators %}
    # {{locator.desc}}
    {{locator.name}} = get_locater('{{page}}','{{locator.name}}')
    {% endfor %}
{% endfor %}